In [1]:
pd.set_option('display.max_colwidth', 50)

src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)



NameError: name 'pd' is not defined

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_files

In [2]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score
from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['retina']
from sklearn.datasets import load_files
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/topper/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/topper/s_training/projects/everylife_model/venv/lib/python3.6/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [3]:
imdb_train = load_files("/Users/topper/s_training/projects/everylife/aclImdb/train")

In [4]:
text_train, y_train = imdb_train.data, imdb_train.target

In [5]:
text_train_df = pd.DataFrame({'text': text_train})

In [6]:
text_train_df.head()

,text
0,"b""Zero Day leads you to think, even re-think w..."
1,b'Words can\'t describe how bad this movie is....
2,b'Everyone plays their part pretty well in thi...
3,b'There are a lot of highly talented filmmaker...
4,b'I\'ve just had the evidence that confirmed m...


# SAMPLING FOR TEST

In [7]:
text_train_df_samp = text_train_df.head(1000)

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['film','movie','picture','review'])

In [77]:
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [96]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Detecting phrases based on collocation counts bi_min=15, tri_min=10
def bigrams(words, bi_min=1, tri_min=1):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [25]:
words = list(sent_to_words(text_train_df_samp.text))
words = remove_stopwords(words)
bigram = bigrams(words)
bigram = [bigram[review] for review in words]

In [26]:
type(bigram)

gensim.models.phrases.Phraser

In [87]:
w = [['sci', 'fi', 'sci','fi','sci','fi','sci', 'fi', 'sci','fi','sci','fi','sci', 'fi', 'sci','fi','sci','fi','sci', 'fi', 'sci','fi','sci','fi','sci', 'fi', 'sci','fi','sci','fi'], ['sci', 'fi', 'sci','fi','sci','fi']]

In [ ]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS


In [ ]:
w

In [98]:
b = bigrams(w)

In [100]:
b = [b[review] for review in w]

In [101]:
b

[['sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi',
  'sci',
  'fi'],
 ['sci', 'fi', 'sci', 'fi', 'sci', 'fi']]

In [79]:
'sci_fi' in b[0]

True

In [17]:
bigram = [bigram[review] for review in words]

In [18]:
bigram

[['zero',
  'day',
  'leads',
  'think',
  'even',
  'think',
  'two',
  'boys',
  'young',
  'men',
  'would',
  'commit',
  'mutual',
  'suicide',
  'via',
  'slaughtering',
  'classmates',
  'captures',
  'must',
  'beyond',
  'bizarre',
  'mode',
  'two',
  'humans',
  'decided',
  'withdraw',
  'common',
  'civility',
  'order',
  'define',
  'mutual',
  'world',
  'via',
  'coupled',
  'destruction',
  'br_br',
  'perfect',
  'given',
  'money',
  'time',
  'filmmaker',
  'actors',
  'remarkable',
  'product',
  'terms',
  'explaining',
  'motives',
  'actions',
  'two',
  'young',
  'suicide',
  'murderers',
  'better',
  'elephant',
  'terms',
  'gets',
  'rationalistic',
  'skin',
  'far',
  'far',
  'better',
  'almost',
  'anything',
  'likely',
  'see',
  'br_br',
  'flawed',
  'honest',
  'terrible',
  'honesty'],
 ['words',
  'describe',
  'bad',
  'explain',
  'writing',
  'see',
  'get',
  'grip',
  'horrible',
  'really',
  'recommend',
  'many',
  'clich_xc',
  'xa',


In [10]:
def get_corpus(df):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [11]:
# N.B. REVERT OFF SAMPLE AFTER TEST
train_corpus4, train_id2word4, bigram_train4 = get_corpus(text_train_df_samp)  

In [12]:
len(train_corpus4)

1000

In [14]:
with open('train_corpus4 + 'pkl', 'wb') as f:
    pickle.dump(train_corpus4, f)
with open('train_id2word4.pkl', 'wb') as f:
    pickle.dump(train_id2word4, f)
with open('bigram_train4.pkl', 'wb') as f:
    pickle.dump(bigram_train4, f)

In [30]:
with open('train_corpus4.pkl', 'rb') as f:
    train_corpus4 = pickle.load(f)
with open('train_id2word4.pkl', 'rb') as f:
    train_id2word4 = pickle.load(f)
with open('bigram_train4.pkl', 'rb') as f:
    bigram_train4 = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'train_corpus4.pkl'

In [13]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus4,
                           num_topics=10,
                           id2word=train_id2word4,
                           chunksize=100,
                           workers=2, # Num. Processing Cores - 1
                           passes=20,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

In [14]:
lda_train4.print_topics(20,num_words=15)[:10]

[(0,
  '0.016*"kids" + 0.012*"de" + 0.011*"disney" + 0.011*"old" + 0.011*"version" + 0.010*"house" + 0.009*"evil" + 0.008*"xc_xa" + 0.008*"london" + 0.007*"classic" + 0.007*"loved" + 0.007*"still" + 0.007*"films" + 0.007*"original" + 0.007*"story"'),
 (1,
  '0.016*"would" + 0.015*"really" + 0.012*"dvd" + 0.012*"funny" + 0.009*"give" + 0.008*"seems" + 0.008*"put" + 0.007*"video" + 0.007*"production" + 0.007*"moon" + 0.007*"got" + 0.007*"instead" + 0.007*"chinese" + 0.006*"seen" + 0.006*"documentary"'),
 (2,
  '0.014*"would" + 0.009*"get" + 0.009*"way" + 0.009*"story" + 0.008*"time" + 0.007*"first" + 0.007*"could" + 0.006*"made" + 0.006*"know" + 0.006*"think" + 0.006*"still" + 0.006*"never" + 0.006*"back" + 0.005*"see" + 0.005*"two"'),
 (3,
  '0.037*"series" + 0.024*"episode" + 0.019*"season" + 0.016*"baby" + 0.015*"sci_fi" + 0.014*"must" + 0.013*"father" + 0.013*"tom" + 0.011*"episodes" + 0.011*"first" + 0.010*"boy" + 0.009*"also" + 0.008*"happened" + 0.008*"love" + 0.008*"favorite"'),


In [19]:
train_vecs = []
for i in range(len(text_train)):
    top_topics = lda_train4.get_document_topics(train_corpus4[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(10)]
    topic_vec.extend([len(text_train_df.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)
    
    

In [123]:
len(text_train)

25000

In [28]:
len(train_vecs)



25000

In [29]:
X = np.array(train_vecs)


In [33]:
y = y_train


In [34]:
with open('y.pkl', 'wb') as f:
    pickle.dump(y, f)
    
with open('X.pkl', 'wb') as f:
    pickle.dump(X, f)

In [41]:
kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')


Logistic Regression Val f1: 0.795 +- 0.006
Logisitic Regression SGD Val f1: 0.789 +- 0.007
SVM Huber Val f1: 0.134 +- 0.268


### Bring in the test data and run against the trained model 

In [42]:
imdb_test = load_files("/Users/topper/s_training/projects/everylife/aclImdb/test")
text_test, y_test = imdb_test.data, imdb_test.target


In [4]:
def importing_test_data():
    imdb_test = load_files("../data/raw/aclImdb/test")
    return imdb_test

ddd = importing_test_data()

In [47]:
text_test_df = pd.DataFrame({'text': text_test})


In [44]:
print(f'No rows in text_test: {len(text_test)}, no of rows in y_test: {len(y_test)}')


No rows in text_test: 25000, no of rows in y_test: 25000


In [110]:
# Bring in train_id2word
with open('train_id2word4.pkl', 'rb') as f:
    train_id2word = pickle.load(f)
    

In [48]:
def get_bigram(df):
    """
    For the test data we only need the bigram data built on the TEST reviews,
    using the TRAIN id2word mappings. 
    """
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    return bigram



In [111]:
bigram_test = get_bigram(text_test_df)


In [55]:
# Load in the trained LDA model
lda_train4 = gensim.models.ldamulticore.LdaMulticore.load('lda_train4.model')



In [119]:
# Using the training dict to create a new corpus
# Corpus - containing the word id and its frequency in every document
test_corpus = [train_id2word.doc2bow(text) for text in bigram_test]


In [107]:
text_test_df.head(1)


,text
0,"b""Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!"""


In [109]:
train_id2word[18652]


'heather_graham'

In [125]:
# Use the model from the TRAINING data and then run through unseen test reviews

test_vecs = []
for i in range(len(text_test)):
    top_topics = lda_train4.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(10)]
    topic_vec.extend([len(text_test_df.iloc[i].text)]) # length review
    test_vecs.append(topic_vec)


In [126]:
len(test_vecs)


25000

In [127]:
len(text_test)

25000

In [128]:
X = np.array(test_vecs)

In [129]:
y = y_test

In [130]:
# X now unseen data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

ss = StandardScaler()
X = ss.fit_transform(X)

lr = LogisticRegression(
  class_weight= 'balanced',
  solver='newton-cg',
  fit_intercept=True
  ).fit(X, y)

y_pred_lr = lr.predict(X)
print(f1_score(y, y_pred_lr,average='binary'))

sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced',shuffle=True
    ).fit(X, y)
    
y_pred_huber = sgd_huber.predict(X)
print(f1_score(y, y_pred_huber, average='binary'))

0.791959157626037
0.0
